### Unsupervised Machine Learning
Assignment #3 <br>
Basic Sequential Clustering Algrorithm <br>
Michael Zwartz<br>

#### Technical Description
Basic Sequential Clustering (BSCA) is a very simple clstering method. Hence the "basic" in its name. In unsupervised machine learning clustering is a important technique to group together instances to sort new data or extract significant features. There are many variants of BSCA that change based on the user defined distance measurment used, threshold of dissimilarity, and maximum # of clusters. 

In [7]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

#load file
data = pd.read_csv('batStats_clean20.csv')

#import LDA and PCA data
from PCAandLDA import batStats1D

#add 1D array to stars dataframe
#stars['PCA1'] = stars1D

#data.head()

In [ ]:
#USER PRARAMETERS DEFINED
#m = max number of clusters set by user
m=3
#a = alpha, threshold of dissimilarity set by user
a=0.9

#t = counter, counts number of iterations or clusters created
t=1

#initialize cluster dictionary
clusters = dict(zip(range(m), [[] for i in range(m)]))

#assign the first data point to cluster 1
clusters[0].append(stars.PCA1[0])

In [ ]:
#iterate through data points after the first one
for row in stars.PCA1[1:]:
#add at least 1 data point to each cluster   
#if counter is less than max clusters aka not all clusters have been instantiated
    c=0 
    currentclusters=t
    distance = dict(zip(range(currentclusters), [[] for i in range(currentclusters)]))
    for c in range(currentclusters):
        distance[c].append(abs(row-np.mean(clusters[c])))
    shortest_dist=min(distance.values())
    closest_cluster=list(distance.keys())[list(distance.values()).index(shortest_dist)]
    if t<m and shortest_dist[0] >a:
        print('cluster ' + str(c+1) + ' is made')
        clusters[currentclusters].append(row)
        t=t+1
#after all clusters have at least one data point sort the rest of the data point by choosing cluster with closest centroid
    else:  
        clusters[closest_cluster].append(row)
print(clusters)

c_df=pd.DataFrame()
for c in clusters:
    for i in clusters[c]:
        c_df=c_df.append({'Value': i, 'Cluster': c}, ignore_index=True)

fig=go.Figure()

fig=px.scatter(x=c_df.Value, color=c_df.Cluster, color_continuous_scale=['red','green','blue'], title="BSCA with 1D PCA Stars Data alpha=" + str(a) + ", max clusters=" + str(m))
fig.update_layout(coloraxis_colorbar=dict(
    title="Clusters",
    tickvals=[0,1,2,3, 4],
    ticktext=["c1","c2","c3","c4", "c5"],
    lenmode="pixels", len=500,
))
#fig.add_trace(go.scatter(x=clusters[1], y=range(len(clusters[0]))))
fig.write_html('result2.html', auto_open=True)